In [ ]:
from init_data import get_data, clean_data
from tune_hyperparams import tune_hyperparams, annualize_pnl_pct
from calculate_indicators import calculate_indicators
import pandas as pd

In [ ]:
stocks = ["^GSPC", "NVDA", "MSFT", "AAPL", "AMZN", "META", "GOOGL", "TSLA", "BRK-B",
          "JPM", "WMT", "LLY", "ORCL", "V", "NFLX", "AVGO", "MA", "XOM", "COST", "JNJ"]

time_periods = [
    {"period": "6mo", "frequency": "1d", "interval": "day"},
    {"period": "1y", "frequency": "1wk", "interval": "week"},
    {"period": "2y", "frequency": "1wk", "interval": "week"}
]

performance = []

In [ ]:
for symbol in stocks:
    for time_period in time_periods:
        period = time_period["period"]
        frequency = time_period["frequency"]
        interval = time_period["interval"]
        # data processing
        raw_financial_data = get_data(symbol, period=period, frequency=frequency)
        if raw_financial_data.empty:
            continue
        cleaned_financial_data = clean_data(raw_financial_data)
        processed_financial_data = calculate_indicators(cleaned_financial_data)
        processed_financial_data.to_csv(f"data/{symbol}_{period}_{frequency}.csv", index = False)


In [ ]:
for i, symbol in enumerate(stocks):
    for time_period in time_periods:
        period = time_period["period"]
        frequency = time_period["frequency"]
        interval = time_period["interval"]
        processed_financial_data = pd.read_csv(f"data/{symbol}_{period}_{frequency}.csv")
        pnl, pnl_pct, sharpe_ratio = tune_hyperparams(processed_financial_data, interval=interval, period=period)
        annualized_pnl_pct = annualize_pnl_pct(pnl_pct, period)
        print("annualized pnl", annualized_pnl_pct)
        performance_metric = {
            "stock" : symbol,
            "time horizon": period,
            "timeframe": frequency,
            "pnl": pnl,
            "annualized pnl percentage": annualized_pnl_pct,
            "sharpe ratio": sharpe_ratio
        } 
        print(performance_metric)
        performance.append(performance_metric)
    if (i + 1) % 5 == 0:
        temp = pd.DataFrame.from_dict(performance[i-4:])
        print(f"metrics from {i + 1} stocks")
        print(temp["annualized pnl percentage"].mean())
        print(temp["sharpe ratio"].mean())


In [ ]:
performance_1y= performance[1:30:3]
len(performance_1y)
print(performance_1y)

In [ ]:
performance_dataframe = pd.DataFrame.from_dict(performance)
final_avg_return = performance_dataframe['annualized pnl percentage'].mean()
final_avg_sharpe = performance_dataframe['sharpe ratio'].mean()
print(final_avg_return)
print(final_avg_sharpe)

In [ ]:
performance_dataframe.columns
df_sorted = performance_dataframe.sort_values(["annualized pnl percentage", 'sharpe ratio'], ascending=False)
df_sorted

In [ ]:
performance_dataframe.to_csv("performance_test_10.csv",index=False)